In [ ]:
!pip install newspaper3k

In [ ]:
import requests
from datetime import datetime
import pytz
import numpy as np
import pandas as pd
from newspaper import Article
import regex as re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
apis=["pub_435954631e5544a4678d9066e8ffa4e737a4e",
      "pub_43596dfe9f3433fb1c42cd8443508fdd5fb68",
      "pub_4359806e437fcb628c31a6aa44b2064234777",
      "pub_435994e8b13a35ee3169152f8886facc0cb46",
      "pub_43601ef177c0d2d0d5afbb97977fc7e90615d",
      "pub_43603d13779573b91f9e64ebce3bf9b2a87d6",
      "pub_43605ed4a7bc8696257e89e37a5c74f4f4150",
      "pub_43606e4e90115eb7fba8972d9e6dd7162c80b",
      "pub_43609f5fe0d9ba8090d4c775f3485a8807e49",
      "pub_43610bf67edce669baf9cc2e6317ba21dc557",
      "pub_43612f6e8b58205389a3a347b32606f6cb9d8",
      "pub_436131b55ee8eb901a6dfb6543e5332a593ba",
      "pub_4361605d402f133d10aa8484ed2d00c238295",
      "pub_43617ecc1cf23836ccb12ae89568cebaf8d58",
      "pub_4361889cfabb533d1c648320d7c8c1b4d975e",
      "pub_43620e3c724184aca5d3ddd997d8c60e5ccb7",
      "pub_43621ee475861ae8e36d1dc796bf5b44bb643",
      "pub_43622f3dfc84284b1eaa4b1f23252101b12f0",
      "pub_43623153d19c6ebccf8d41418824c128333e8",
      "pub_436240f8072416d11b7d0d709179ba2737b34"]

In [ ]:
articles_list = []
page=None

for api in apis:
  if page is None:
    url = f"https://newsdata.io/api/1/news?apikey={api}&country=in&language=en"
    data = requests.get(url).json()
  else:
    url = f"https://newsdata.io/api/1/news?apikey={api}&country=in&language=en&page={page}"
    data = requests.get(url).json()

  if data["status"]=="success":
    for article in data['results']:
      id = article['article_id']
      source = article['source_id']
      category = article['category']
      title = article['title']
      link = article['link']
      pubDate = article['pubDate']
      ist_time = datetime.strptime(pubDate, "%Y-%m-%d %H:%M:%S").astimezone(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S")
      articles_list.append({'ID': id, 'Source': source, 'Category': category, 'Title': title, 'Link': link, 'PubDate': ist_time})

  page=data["nextPage"]

  while True:
    url = f"https://newsdata.io/api/1/news?apikey={api}&country=in&language=en&page={page}"
    data = requests.get(url).json()

    if data["status"]!="success":
      break

    for article in data['results']:
      id = article['article_id']
      source = article['source_id']
      category = article['category']
      title = article['title']
      link = article['link']
      pubDate = article['pubDate']
      ist_time = datetime.strptime(pubDate, "%Y-%m-%d %H:%M:%S").astimezone(pytz.timezone('Asia/Kolkata')).strftime("%Y-%m-%d %H:%M:%S")
      page=data["nextPage"]
      status=data["status"]
      articles_list.append({'ID': id, 'Source': source, 'Category': category, 'Title': title, 'Link': link, 'PubDate': ist_time})

df = pd.DataFrame(articles_list)

In [ ]:
df['PubDate'] = pd.to_datetime(df['PubDate'])
df = df.sort_values(by='PubDate')

In [ ]:
def extract_text(row):
    try:
        article = Article(row['Link'])
        article.download()
        article.parse()
        text = article.text
        return text
    except:
        return None
df['Body'] = df.apply(extract_text, axis=1)

/usr/local/lib/python3.10/dist-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname IST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [ ]:
df = df.dropna()

In [ ]:
def cleaning(text):
  cleaned_text = text.replace('\\n'," ")                              # removal of new line symbol \n

  cleaned_text = re.sub(r'\s+'," ",cleaned_text)                      # Removal of additional white space

  return cleaned_text

df['Body']=df['Body'].apply(cleaning)

In [ ]:
def count_words(text):
    if text is not None:
        return len(text.split())
    else:
        return None
df['Number of Words'] = df['Body'].apply(count_words)

In [ ]:
df = df[df['Number of Words'] > 100]

In [ ]:
df.to_csv("/content/drive/MyDrive/data.csv",index=False)